# Decorators

* Functions
* Wrap other functions and enhance their behavior
* Examples of higher order functions
* Have their own syntax, using "@" (syntactic sugar)

An example without the syntactic sugar.

Below, `wrapper()` is the higher order function that gets returned in `be_polite(fn)`…

In [23]:
def be_polite(fn):
    def wrapper():
        print("What a pleasure to meet you!")
        fn()
        print("Have a great day!")
    return wrapper

def greet():
    print("My name is Colt.")

def rage():
    print("I HATE YOU!!!")

This is where we decorate it, the syntactic sugar will normally be used instead of lines like below.

In [25]:
my_greet = be_polite(greet)

polite_rage = be_polite(rage)

In [26]:
my_greet()

What a pleasure to meet you!
My name is Colt.
Have a great day!


In [27]:
polite_rage()

What a pleasure to meet you!
I HATE YOU!!!
Have a great day!


The normal way to use a decorator is with the syntactic sugar "@" like so…

In [29]:
def be_polite(fn):
    def wrapper():
        print("What a pleasure to meet you!")
        fn()
        print("Have a great day!")
    return wrapper

@be_polite
def greet():
    print("My name is Matt.")

@be_polite
def rage():
    print("SHADDAP JERKY!!!")

### Explanation:

* You'll call `greet()` or `rage()`
* It'll pass the output from either of those functions as called into the decorator `be_polite(fn)` as the `fn` argument
* The output from `greet()` or `rage()` gets piped into the `wrapper()` higher order function inside `be_polite()`

In [30]:
greet()

What a pleasure to meet you!
My name is Matt.
Have a great day!


In [31]:
rage()

What a pleasure to meet you!
SHADDAP JERKY!!!
Have a great day!


## Decorators with Different Signatures

**NOTE:** `wrapper()` as the higher order function is just a standard. It can actually be any name as long as it gets returned in the end of the decorator function.

In [44]:
def shout(fn):
    def wrapper(*args, **kwargs):
        return fn(*args, **kwargs).upper()
    return wrapper

@shout
def greet(name):
    return f"Hi, I'm {name}."

@shout
def order(main, side):
    return f"Hi, I'd like the {main}, with a side of {side}, please."

@shout
def lol():
    return "lol"

One argument passed, we've seen it before…

In [45]:
greet("Todd")

"HI, I'M TODD."

Two arguments this time, the get passed into `*args` in `wrapper()`

In [46]:
order("burger", "fries")

"HI, I'D LIKE THE BURGER, WITH A SIDE OF FRIES, PLEASE."

Or even no arguments passed…

In [47]:
lol()

'LOL'

`**kwargs` will be activated if you pass named arguments…

In [48]:
order(side="mashed potatoes", main="fishwich")

"HI, I'D LIKE THE FISHWICH, WITH A SIDE OF MASHED POTATOES, PLEASE."

## Preserve Metadata

In [63]:
def log_function_data(fn):
    def wrapper(*args, **kwargs):
        """I AM A WRAPPER FUNCTION"""
        print(f"you are about to call: {fn.__name__}")
        print(f"here's the documentation: {fn.__doc__}")
        return fn(*args, **kwargs)
    return wrapper

@log_function_data
def add(x, y):
    """Adds two numbers together"""
    return x + y

In [64]:
print(add(10, 30))

you are about to call: add
here's the documentation: Adds two numbers together
40


But note what happens when call dunders on the add function…

In [65]:
print(add.__doc__)

I AM A WRAPPER FUNCTION


In [66]:
print(add.__name__)

wrapper


To remedy this, you'd use the `functools` package…

```python
from functools import wraps

def my_decorator(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        # do something here
        pass
    return wrapper
```

In [67]:
from functools import wraps

def log_function_data(fn):
    @wraps(fn) # <-- this is all you need to add
    def wrapper(*args, **kwargs):
        """I AM A WRAPPER FUNCTION"""
        print(f"you are about to call: {fn.__name__}")
        print(f"here's the documentation: {fn.__doc__}")
        return fn(*args, **kwargs)
    return wrapper

@log_function_data
def add(x, y):
    """Adds two numbers together"""
    return x + y

In [68]:
print(add.__doc__)

Adds two numbers together


In [69]:
print(add.__name__)

add


In [70]:
help(add)

Help on function add in module __main__:

add(x, y)
    Adds two numbers together



## Speed-Test Decorator

Use decorator to make the speed tests written in the Generators section of the course easier to call.

In [12]:

def speed_test(fn):
    from functools import wraps
    from time import time
    
    @wraps(fn)
    def wrapper(*args, **kwargs):
        start_time = time()
        result = fn(*args, **kwargs)
        end_time = time()
        rounded_time = round(end_time - start_time, 2)
        print(f"Time elapsed: {rounded_time} seconds")
        return result
    
    return wrapper

@speed_test
def sum_gen():
    return sum(x for x in range(10000000))

@speed_test
def sum_list():
    return sum([x for x in range(10000000)])

sum_gen()

Time elapsed: 0.56 seconds


49999995000000

In [13]:
sum_list()

Time elapsed: 0.68 seconds


49999995000000

## Exercise 89: show_args

Write a function called **show_args** which accepts a function and returns another function. Before invoking the function passed to it, **show_args** should be responsible for printing two things: a tuple of the positional arguments, and a dictionary of the keyword arguments.

In [20]:
from functools import wraps

def show_args(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        print(f"args: {args}")
        print(f"kwargs: {kwargs}")
        return fn(*args, **kwargs)
    return wrapper

@show_args
def do_nothing(*args, **kwargs):
    print()
    print(f"Output from decorated function: {kwargs['a']}")

do_nothing(1, 2, 3, a="hi", b="bye")


args: (1, 2, 3)
kwargs: {'a': 'hi', 'b': 'bye'}

Output from decorated function: hi


## Example: Ensuring Args with a Decorator

In [23]:
def ensure_no_kwargs(fn):
    from functools import wraps
    @wraps(fn)
    def wrapper(*args, **kwargs):
        if kwargs:
            raise ValueError("No kwargs allowed! Sorry.")
        return fn(*args, **kwargs)
    return wrapper

@ensure_no_kwargs
def greet(name):
    print(f"Hi there {name}")

greet("Tony")


Hi there Tony


In [25]:
# this should error out with the
# custom error message
greet(first="Rob", last="Kistner")


ValueError: No kwargs allowed! Sorry.

## Exercise 90: double_return

Write a function called **double_return** which accepts a function and returns another function, **double_return** should decorate a function by returning two copies of the inner function's return value inside a list.

In [29]:
def double_return(fn):
    from functools import wraps
    @wraps(fn)
    def wrapper(*args, **kwargs):
        return [fn(*args, **kwargs), fn(*args, **kwargs)]
    return wrapper

@double_return
def add(x, y):
    return x+y

@double_return
def greet(name):
    return "Hi, I'm " + name

greet("Colt")


["Hi, I'm Colt", "Hi, I'm Colt"]

## Exercise 91: ensure_fewer_than_three_args

Write a function called **ensure_fewer_than_three_args** which accepts a function and returns another function. The function passed to it should only be invoked if there are fewer than three positional arguments passed to it. Otherwise, the inner function should return "Too many arguments".

In [35]:
def ensure_fewer_than_three_args(fn):
    from functools import wraps
    @wraps(fn)
    def wrapper(*args, **kwargs):
        if len(args) < 3:
            return fn(*args, **kwargs)
        return "Too many arguments!"
    return wrapper

@ensure_fewer_than_three_args
def add_all(*args, **kwargs):
    result = 0
    for n in args:
        result += n
    return result

add_all(3,2,1)


'Too many arguments!'

In [36]:
add_all(3,2)

5

## Exercise 92: only_ints

Write a function called **only_ints** which accepts a function and returns another function. The function passed to it should only be invoked if all the arguments passed to it are integers. Otherwise it should return "Please only invoke with integers".

In [62]:
def only_ints(fn):
    from functools import wraps
    @wraps(fn)
    def wrapper(*args, **kwargs):
        if any([arg for arg in args if type(arg) != int]):
            return "Please only invoke with integers"
        return fn(*args, **kwargs)
    return wrapper

@only_ints
def add(*args):
    print("Output from add() function")

add(1, 2)


Output from add() function


In [63]:
add("1", 2)

'Please only invoke with integers'

## Exercise 93: ensure_authorized

Write a function called **ensure_authorized** which accepts a function and returns another function. The function passed to it should only be invoked if there is a keyword argument named "role" with a value of "admin", otherwise, the inner function should return "Unauthorized".

In [68]:
def ensure_authorized(fn):
    from functools import wraps
    @wraps(fn)
    def wrapper(*args, **kwargs):
        if kwargs.get("role") == "admin":
            return fn(*args, **kwargs)
        return "Unauthorized"
    return wrapper

@ensure_authorized
def show_secrets(*args, **kwargs):
    return "Shh! Don't tell anybody!"

show_secrets(role="admin")


"Shh! Don't tell anybody!"

In [69]:
show_secrets(role="nobody")

'Unauthorized'

In [70]:
show_secrets()

'Unauthorized'

## Example: writing an ensure_first_arg_is decorator

We want to pass an argument into the decorator `wrapper` function

In [73]:
def ensure_first_arg_is(val):
    from functools import wraps
        # this intermediate inner function will run as the 
        # higher order function, then wrapper acts as the 
        # function that receives the required argument(s)
    def inner(fn):
        @wraps(fn)
        def wrapper(*args, **kwargs):
            if args and args[0] != val:
                return f"First arg needs to be {val}"
            return fn(*args, **kwargs)
        return wrapper
    return inner
        
@ensure_first_arg_is("burrito")
def fav_foods(*foods):
    print(foods)

fav_foods("burrito", "ice cream")


('burrito', 'ice cream')


In [74]:
fav_foods("pizza", "asparagus")

'First arg needs to be burrito'

## Example: Enforcing Argument Types with a Decorator

In [80]:
def enforce(*types):
    def decorator(f):
        def new_func(*args, **kwargs):
            newargs = []
            for (a, t) in zip(args, types):
                # this is where the vars get cast
                # the proper types
                newargs.append(t(a))
            return f(*newargs, **kwargs)
        return new_func
    return decorator

@enforce(str, int)
def repeat_msg(msg, times):
    for time in range(times):
        print(msg)

hello
hello
hello


For the below call…

zipped:
`("hello", strs) ("3", int)`

then convert ("hello", to a string)

then convert ("3", to an int)

`["hello", 3]`


In [81]:
repeat_msg("hello", "3")


hello
hello
hello


## Example: Delay Decorator

In [86]:
 
def delay(timer):
    from functools import wraps
    from time import sleep

    def inner(fn):
    
        @wraps(fn)
        def wrapper(*args, **kwargs):
            print("Waiting {}s before running {}".format(timer, fn.__name__))
            sleep(timer)
            return fn(*args, **kwargs)
        return wrapper
    
    return inner

@delay(3)
def show_it():
    print(f"Showing it after a delay.")

show_it()


Waiting 3s before running show_it
Showing it after a delay.
